# Proyecto: Informe de Emails Recibidos

## Introducción

### Problema a abordar

La problemática radica en la gran cantidad de mails que recibimos todos los días. En esa marea de mails, se suelen pasar por alto mails importantes. Muchas veces, un mail queda perdido en el buzón de entrada y luego se hace difícil encontrarlo. Si además se refería a algo urgente que necesitaba verse a tiempo, se torna un problema mayor.

### Desarrollo de la propuesta de solución

La propuesta es generar un informe de envío semanal a una casilla de correo que haga un repaso de todos los mails recibidos en la última semana, de manera que nos permita ponernos al día con los asuntos a atender. Se buscará que el informe ordene los temas relevados por categorías según los mails recibidos.
La idea consiste en acceder a un mail en Outlook o exportar los mails de la última semana y a través de un prompt leer los mails en cuestión y armar un resumen de estos ordenándolos por categoría y prioridad.

**Prompt a probar:** Analizar todos los emails de la última semana (remitente, asunto, contenido y adjuntos) y generar un informe agrupando por categoría. Identificar los mails de mayor prioridad y proponer acciones a llevar a cabo en la semana.

**Prompt más estructurado:**

```text
Eres un asistente que analiza correos de Outlook.  
Responde en español, claro y estructurado, en menos de 200 palabras.  

IMPORTANTE: Oculta datos sensibles reemplazando los nombres de personas por [REDACTED] y las direcciones de correo electrónico por [MAIL].  
No inventes ni modifiques información, solo redacta lo sensible.  

Entrega:
1) Panorama (tendencias y temas) en 5-8 bullets.  
2) Top prioridades (máx 10) con acción sugerida.  
3) Fechas/calendario detectados (si se mencionan).  
4) Adjuntos relevantes (asunto).  
5) KPIs: total, sin leer, con adjuntos, respondidos.  

```

Para usar también el modelo texto a imagen, buscaremos que el informe comience con un tablero que muestre toda la información resumida de manera muy visual.


### Viabilidad del proyecto
El proyecto lo podemos dividir en 3 etapas:

#### 1) Obtener mails

Primero necesitamos tener el contenido de los correos electrónicos recibidos en una casilla de correo. Para esto, se buscará acceder con la API de OpenAI a la bandeja de entrada de Outlook o automatizar la exportación de los mails de la última semana para luego poder entregarle el archivo a OpenAI. Esta etapa es la que a priori resulta la más desafiante con los conocimientos que poseo, pero utilizando ChatGPT es viable encontrar una manera de realizarlo.

#### 2) Generar informe

La segunda etapa sería cuando entra en juego los modelos de ChatGPT y Dall-E para analizar todos los mails y armar el informe. En esta etapa, se necesitará pulir el prompt en ambos modelos para lograr el objetivo.

#### 3) Enviar informe

La tercera y última etapa consiste en enviar por email el informe generado. Por más que actualmente no cuento con los conocimientos para realizarlo, utilizando ChatGPT es viable encontrar como se puede llevar a cabo.

## Objetivos

Sistematizar la lectura de correos para transformar información dispersa en un informe ejecutivo claro y accionable.

Estandarizar criterios de priorización y seguimiento para mejorar la toma de decisiones.

Proteger la privacidad mediante anonimización constante de datos personales.

## Metodología

El flujo implementado debe convertir un alto volumen de correos en un producto informativo breve, verificable y anonimizado, apto para lectura ejecutiva y para integrar a tableros/automatizaciones.

Las 3 etapas mencionadas se llevan a cabo de la siguiente manera:

**Etapa 1: Obtener mails (Recolección y depuración)**

- Obtengo correos desde Microsoft Graph (Outlook) del período de análisis y se eliminan duplicidades, ruidos y datos superfluos, preservando la trazabilidad.
- Normalizo campos (remitente, asunto, cuerpo, fechas, flags/“Destacado”, adjuntos, webLink). Excluyo correos de promociones, newsletters y enviados por mí salvo que sean relevantes (respuestas a clientes).

**Etapa 2: Generar informe (Diseño de prompt y orquestación)**

- Uso instrucciones de sistema claras (rol, estilo, longitud máxima, idioma) y secciones obligatorias (Panorama, Top prioridades con acción, Fechas, Adjuntos, KPIs) para producir un resumen ejecutivo con secciones fijas de menos de 200 palabras.
- Aplico anonimización para reemplazar emails y nombres propios por placeholders antes de enviar al modelo y también en la salida. 
- Se verifican indicadores básicos (totales, no leídos, adjuntos, respondidos) para asegurar calidad.

**Etapa 3: Enviar Informe**

- Genero PDF profesional del informe (prioridades, fechas, adjuntos).
- Automatización y monitoreo: Ejecución programada (Power Automate).

## Herramientas y tecnologías

Se utilizan:
- Visual Studio Code
- Jupyter Notebook
- Github

- Lenguaje: **Python**
- Modelo AI: **Google Gemini**

**Tecnica de Prompt:**

Se busca utilizar en primera instancia la tecnica zero-shot dado que la cantidad de tokens va a ser alta de por si por la cantidad de correos que tiene que analizar el modelo. Se trata de optimizar el consumo de tokens. De igual manera, se evaluará si el resultado obtenido es acorde a lo esperado y, en caso de no serlo, se optará por agregar ejemplos (few-shot) para obtener respuestas breves y bien formateadas.

## Implementación

A continuación se incluye el código para llegar a la solución propuesta:

In [1]:
import os, json
from datetime import datetime
from dotenv import load_dotenv
from google import genai
from google.genai import types  # <- para GenerateContentConfig

# =========================
# Config
# =========================
load_dotenv()

# En el SDK nuevo no hace falta el prefijo "models/"
# Si lo traés así desde el entorno, se lo quitamos para evitar 404.
_MODEL_ENV = os.getenv("GOOGLE_GEMINI_MODEL", "gemini-2.5-flash")
MODEL = _MODEL_ENV.replace("models/", "")
# El cliente toma GEMINI_API_KEY o GOOGLE_API_KEY desde el entorno
client = genai.Client()

# === Config única (editar solo acá) ===
CONFIG = {
    "from_mailreader": True,
    "limit": 200, #limite de mails a buscar
    "days_back": 30, #cantidad de dias hacia atras para buscar mails
    "json_path": None,
    "max_chars": 120_000,
    "out": None,                 # "respuesta.txt" o None
    "model": MODEL,              # o "gemini-2.5-pro", etc.
    "temperature": None,         # ej. 0.3
    "max_output_tokens": None,   # ej. 1200
    "response_mime_type": None,  # ej. "text/markdown"
    "debug": False,
}


# =========================
# Helpers
# =========================

def load_rows_from_json(path: str) -> list[dict]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, dict) and "rows" in data:
        return data["rows"]
    if not isinstance(data, list):
        raise ValueError("El JSON no es una lista de correos.")
    return data

def load_rows_from_mailreader(limit: int | None = None, days_back: int | None = None) -> list[dict]:
    """
    Importa mailreader13 y obtiene los correos en memoria.
    Cambiá 'mailreader13' si tu módulo se llama diferente.
    """
    try:
        import mailreader13_beta as mailreader
    except Exception as e:
        raise RuntimeError(f"No pude importar mailreader13: {e}")

    if hasattr(mailreader, "collect_rows"):
        # Pasamos limit/days_back si tu collect_rows los soporta
        try:
            rows = mailreader.collect_rows(limit=limit, days_back=days_back or None, debug=False)
        except TypeError:
            # fallback por si tu firma solo acepta limit
            rows = mailreader.collect_rows(limit=limit)
    elif hasattr(mailreader, "main"):
        out_path = os.getenv("OUTPUT_JSON", "emails.json")
        mailreader.main()
        rows = load_rows_from_json(out_path)
    else:
        raise RuntimeError("mailreader no expone collect_rows() ni main().")

    return rows

def clamp_rows(rows: list[dict], max_chars: int = 120_000) -> list[dict]:
    """Devuelve filas *compactas* y recorta por caracteres de forma consistente."""
    compact_rows, total = [], 0
    for r in rows:
        compact = {
            "Fecha": r.get("Fecha"),
            "ReceivedUTC": r.get("ReceivedUTC"),
            "Remitente": r.get("Remitente"),
            "RemitenteNombre": r.get("RemitenteNombre"),
            "Asunto": r.get("Asunto"),
            "Cuerpo": (r.get("Cuerpo") or "")[:2000],  # límite duro por mail
            "TieneAdjuntos": bool(r.get("TieneAdjuntos")),
            "Leido": bool(r.get("Leido")),
            "Respondido": bool(r.get("Respondido")),
            "Categorias": r.get("Categorias"),
            "WebLink": r.get("WebLink"),
        }
        s = json.dumps(compact, ensure_ascii=False)
        if total + len(s) > max_chars and compact_rows:
            break
        compact_rows.append(compact)
        total += len(s)
    return compact_rows

def build_prompt(rows: list[dict]) -> tuple[str, list[str]]:
    """
    Devuelve (system_instruction, contents_list) en el formato esperado por google-genai.
    - system_instruction: string con las reglas
    - contents_list: lista[str] con el payload (el SDK la convierte a UserContent)
    """
    system_instruction = (
        "Eres un asistente que analiza correos de Outlook. "
        "Responde en español, claro y estructurado, en menos de 200 palabras."
        "IMPORTANTE: Oculta datos sensibles reemplazando los nombres de personas por [REDACTED] y las direcciones de correo electronico por [MAIL]. "
        "No inventes ni modifiques información, solo redacta lo sensible. Entrega:\n"
        "1) Panorama (tendencias y temas) en 5-8 bullets.\n"
        "2) Top prioridades (máx 10) con acción sugerida.\n"
        "3) Fechas/calendario detectados (si se mencionan).\n"
        "4) Adjuntos relevantes (asunto).\n"
        "5) KPIs: total, sin leer, con adjuntos, respondidos.\n"
        f"Fecha de generación: {datetime.now().isoformat(timespec='seconds')}."
    )

    contents_list = [
        "A continuación va la lista JSON de correos.",
        json.dumps(rows, ensure_ascii=False),
    ]
    return system_instruction, contents_list


# =========================
# Ejecutor
# =========================
def run_analysis(config: dict):
    """Ejecuta el análisis leyendo TODO de config."""
    from_mailreader     = config["from_mailreader"]
    limit               = config["limit"]
    days_back           = config["days_back"]
    json_path           = config["json_path"]
    max_chars           = config["max_chars"]
    out                 = config["out"]
    model               = (config["model"] or MODEL).replace("models/", "")
    temperature         = config["temperature"]
    max_output_tokens   = config["max_output_tokens"]
    response_mime_type  = config["response_mime_type"]
    debug               = config["debug"]

    # 1) Carga de correos
    if from_mailreader:
        rows = load_rows_from_mailreader(limit=limit, days_back=days_back)
    else:
        jp = json_path or os.getenv("OUTPUT_JSON", "emails.json")
        rows = load_rows_from_json(jp)

    if not rows:
        return "", {"error": "No hay correos para analizar."}

    # 2) Orden + recorte
    rows_sorted  = sorted(rows, key=lambda r: r.get("ReceivedUTC") or r.get("Fecha") or "", reverse=True)
    rows_compact = clamp_rows(rows_sorted, max_chars=max_chars)

    # 3) Prompt
    system_instruction, contents = build_prompt(rows_compact)

    # 4) Llamada al modelo
    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction,
                max_output_tokens=max_output_tokens if max_output_tokens else None,
                temperature=temperature if temperature else None,
                response_mime_type=response_mime_type if response_mime_type else None,
            ),
        )
    except Exception as e:
        if debug:
            raise
        return "", {"error": f"Fallo generate_content: {e}", "model": model}

    text = getattr(response, "text", "") or ""

    # 5) Guardado opcional
    saved_path = None
    if out:
        saved_path = os.path.abspath(out)
        with open(out, "w", encoding="utf-8") as f:
            f.write(text)

    # 6) Métricas de uso
    usage = {}
    if hasattr(response, "usage_metadata") and response.usage_metadata:
        u = response.usage_metadata
        usage = {
            "prompt_tokens": getattr(u, "prompt_token_count", None),
            "response_tokens": getattr(u, "candidates_token_count", None),
            "thoughts_tokens": getattr(u, "thoughts_token_count", None),
            "total_tokens": getattr(u, "total_token_count", None),
        }

    meta = {
        "model": model,
        "saved_to": saved_path,
        "usage": usage,
        "rows_in": len(rows),
        "rows_after_clamp": len(rows_compact),
    }
    return text, meta


# =========================
# Invocación
# =========================
texto, meta = run_analysis(CONFIG)
print(texto)
meta

Aquí tienes el análisis de tus correos:

**1) Panorama (tendencias y temas):**
*   Gestión de pagos y vencimientos de autónomos.
*   Actualizaciones y seguimiento de tareas contables y financieras.
*   Comunicación y agradecimiento por soporte en gestión de proveedores.
*   Noticias sobre nuevas herramientas tecnológicas (GPT-5 en Copilot).
*   Asesoramiento legal sobre gestión de fallecimiento de empleados.
*   Solicitud y envío de documentación impositiva.
*   Confirmación de operaciones financieras (venta de bonos).

**2) Top prioridades con acción sugerida:**
1.  **Pago Autónomos 08-2025:** Verificar débito o realizar pago del VEP (Vence 08-09-2025).
2.  **Gestión de Proveedores:** Seguir la actualización del sistema de pendientes por [REDACTED].
3.  **Revisiones Contables:** Confirmar con [REDACTED] estado de cuenta USD Allaria y coordinar con [REDACTED] por asientos mensuales.
4.  **Asuntos de Dividendos:** Consultar con [REDACTED] por revisión de planillas y actas.
5.  **Documen

{'model': 'gemini-2.5-flash',
 'saved_to': None,
 'usage': {'prompt_tokens': 4442,
  'response_tokens': 492,
  'thoughts_tokens': 2189,
  'total_tokens': 7123},
 'rows_in': 8,
 'rows_after_clamp': 8}

### Aclaración y pasos siguientes

Cabe aclarar, que en esta primera instancia de proyecto se busca implementar las etapas 1 y 2. La etapa 3 se integrará luego en el proyecto final. Por tal motivo, también queda pendiente la utlizacion del modelo de texto a imagen.

**Optimización:** Ante la gran cantidad de tokens utilizados, luego de comprobar que el codigo funcione, se buscará una manera de optimizarlo para asi lograr una mejor versión final.